In [2]:
import numpy as np
import tifffile
from scipy import ndimage
from skimage.filters import gaussian
from skimage.segmentation import watershed
from skimage.morphology import disk, dilation, erosion
import os
import glob
import re
from google.colab import drive
from tqdm.notebook import tqdm

# Mount Google Drive
drive.mount('/content/drive')

# Define input and output paths
cadherin_dir = '/content/drive/MyDrive/knowledge/University/Master/Thesis/Projected/flow3-x20/Cadherins/tophat'
fused_dir = '/content/drive/MyDrive/knowledge/University/Master/Thesis/Segmented/flow3-x20/Seed'
output_dir = '/content/drive/MyDrive/knowledge/University/Master/Thesis/Segmented/flow3-x20/Cell'

# Create output directory if it doesn't exist
os.makedirs(output_dir, exist_ok=True)

# Find all Cadherin .tif files - UPDATED pattern to match actual filenames
cadherin_files = glob.glob(os.path.join(cadherin_dir, '*_regional_tophat.tif'))
print(f"Found {len(cadherin_files)} Cadherin files to process")

# Function to extract the sequence prefix from filenames
def extract_sequence_prefix(filename):
    # The pattern matches the sequence identifier (e.g., denoised_0Pa_A1_19dec21_40x_L2RA_FlatA_seq005)
    match = re.match(r"(denoised_.*?seq\d+)", os.path.basename(filename))
    if match:
        return match.group(1)
    return None

# Recursively search for all fused cell mask files (including in subfolders)
fused_files = []
for root, dirs, files in os.walk(fused_dir):
    for file in files:
        if file.endswith('_segmented_cells.tif'):
            fused_files.append(os.path.join(root, file))

print(f"Found {len(fused_files)} fused cell mask files")

# Create a dictionary mapping sequence prefixes to fused file paths
fused_prefix_to_file = {}
for fused_file in fused_files:
    prefix = extract_sequence_prefix(fused_file)
    if prefix:
        fused_prefix_to_file[prefix] = fused_file
        print(f"Mapped prefix '{prefix}' to file: {os.path.basename(fused_file)}")

# Process each Cadherin file
for cadherin_file in tqdm(cadherin_files):
    # Get base filename
    filename = os.path.basename(cadherin_file)

    # Extract sequence prefix from the cadherin filename
    prefix = extract_sequence_prefix(cadherin_file)
    if not prefix:
        print(f"WARNING: Could not extract sequence prefix from {filename}, skipping")
        continue

    # Find corresponding fused cell mask file
    if prefix not in fused_prefix_to_file:
        print(f"WARNING: No matching fused cell mask found for {prefix}, skipping")
        continue

    fused_mask_file = fused_prefix_to_file[prefix]
    base_name = prefix

    print(f"\nProcessing: {filename}")
    print(f"Using fused cell mask: {os.path.basename(fused_mask_file)}")

    try:
        # Load the images
        cadherin_img = tifffile.imread(cadherin_file)
        fused_cell_masks = tifffile.imread(fused_mask_file)

        print(f"  Cadherin image shape: {cadherin_img.shape}")
        print(f"  Fused cell mask shape: {fused_cell_masks.shape}")

        # Extract Cadherin channel if needed
        if len(cadherin_img.shape) == 2:
            # Single channel image (already Cadherin)
            print("  Detected single-channel Cadherin image")
            membrane_channel = cadherin_img
        elif len(cadherin_img.shape) == 3 and cadherin_img.shape[0] == 3:
            # Format is (C, H, W)
            print("  Detected format: (C, H, W)")
            membrane_channel = cadherin_img[0]  # First channel
        elif len(cadherin_img.shape) == 3 and cadherin_img.shape[2] == 3:
            # Format is (H, W, C)
            print("  Detected format: (H, W, C)")
            membrane_channel = cadherin_img[:, :, 0]  # First channel
        else:
            print(f"  Unexpected image shape: {cadherin_img.shape}. Using first channel/plane.")
            if len(cadherin_img.shape) == 3:
                membrane_channel = cadherin_img[0] if cadherin_img.shape[0] < cadherin_img.shape[1] else cadherin_img[:, :, 0]
            else:
                membrane_channel = cadherin_img

        # Apply Gaussian blur to reduce noise
        print("  Applying Gaussian blur...")
        membrane_smoothed = gaussian(membrane_channel, sigma=1)

        # Calculate morphological gradient
        print("  Calculating morphological gradient...")
        selem = disk(1)  # Adjust radius if needed
        dilated = dilation(membrane_smoothed, selem)
        eroded = erosion(membrane_smoothed, selem)
        membrane_gradient = dilated - eroded

        # Normalize gradient to 0-1 range
        membrane_gradient_norm = (membrane_gradient - membrane_gradient.min()) / (
                    membrane_gradient.max() - membrane_gradient.min())

        # Apply watershed segmentation using fused cell masks as seeds
        print("  Performing watershed segmentation...")
        watershed_output = watershed(membrane_gradient_norm, fused_cell_masks, mask=membrane_gradient_norm > 0)

        # Save the cell mask result
        cell_mask_path = os.path.join(output_dir, f"{base_name}_cell_mask.tif")
        tifffile.imwrite(cell_mask_path, watershed_output.astype(np.uint32))
        print(f"  Saved cell mask to {cell_mask_path}")

        # Print statistics
        print(f"  Number of fused cell seeds: {len(np.unique(fused_cell_masks)) - 1}")
        print(f"  Number of segmented cells: {len(np.unique(watershed_output)) - 1}")
        print(f"  Processing complete for {filename}")

    except Exception as e:
        print(f"ERROR processing {filename}: {str(e)}")
        continue

print("All processing complete!")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Found 8 Cadherin files to process
Found 5 fused cell mask files


  0%|          | 0/8 [00:00<?, ?it/s]

All processing complete!


In [ ]:
import numpy as np
import tifffile
from scipy import ndimage
from skimage.filters import gaussian
from skimage.segmentation import watershed
from skimage.morphology import disk, dilation, erosion
from skimage.transform import resize
import os
import glob
import re
from google.colab import drive
from tqdm.notebook import tqdm

# Mount Google Drive
drive.mount('/content/drive')

# Define input and output paths
cadherin_dir = '/content/drive/MyDrive/knowledge/University/Master/Thesis/Projected/1.4Pa-x20/Cadherins/tophat'
fused_dir = '/content/drive/MyDrive/knowledge/University/Master/Thesis/Segmented/1.4Pa-x20/Seed_or'
holes_dir = '/content/drive/MyDrive/knowledge/University/Master/Thesis/Segmented/1.4Pa-x20/Holes_masks'
output_dir = '/content/drive/MyDrive/knowledge/University/Master/Thesis/Segmented/1.4Pa-x20/Cell'

# Create output directory if it doesn't exist
os.makedirs(output_dir, exist_ok=True)

# Find all Cadherin .tif files
cadherin_files = glob.glob(os.path.join(cadherin_dir, '*_Cadherins_regional_tophat.tif'))
print(f"Found {len(cadherin_files)} Cadherin files to process")

# Function to extract the sequence prefix from filenames
def extract_sequence_prefix(filename):
    match = re.match(r"(denoised_.*?seq\d+)", os.path.basename(filename))
    if match:
        return match.group(1)
    return None

# Recursively search for all fused cell mask files (including in subfolders)
fused_files = []
for root, dirs, files in os.walk(fused_dir):
    for file in files:
        if file.endswith('_segmented_cells.tif'):
            fused_files.append(os.path.join(root, file))

print(f"Found {len(fused_files)} fused cell mask files")

# Find all hole mask files
hole_files = glob.glob(os.path.join(holes_dir, '*_Cadherins_regional*.tif')) #_segmented
print(f"Found {len(hole_files)} hole mask files")

# Create dictionaries mapping sequence prefixes to file paths
fused_prefix_to_file = {}
for fused_file in fused_files:
    prefix = extract_sequence_prefix(fused_file)
    if prefix:
        fused_prefix_to_file[prefix] = fused_file
        print(f"Mapped prefix '{prefix}' to fused file: {os.path.basename(fused_file)}")

hole_prefix_to_file = {}
for hole_file in hole_files:
    prefix = extract_sequence_prefix(hole_file)
    if prefix:
        hole_prefix_to_file[prefix] = hole_file
        print(f"Mapped prefix '{prefix}' to hole file: {os.path.basename(hole_file)}")

# Process each Cadherin file
for cadherin_file in tqdm(cadherin_files):
    # Get base filename
    filename = os.path.basename(cadherin_file)

    # Extract sequence prefix from the cadherin filename
    prefix = extract_sequence_prefix(cadherin_file)
    if not prefix:
        print(f"WARNING: Could not extract sequence prefix from {filename}, skipping")
        continue

    # Find corresponding fused cell mask file
    if prefix not in fused_prefix_to_file:
        print(f"WARNING: No matching fused cell mask found for {prefix}, skipping")
        continue

    fused_mask_file = fused_prefix_to_file[prefix]

    # Find corresponding hole mask file
    hole_mask_file = None
    if prefix in hole_prefix_to_file:
        hole_mask_file = hole_prefix_to_file[prefix]

    base_name = prefix

    print(f"\nProcessing: {filename}")
    print(f"Using fused cell mask: {os.path.basename(fused_mask_file)}")
    if hole_mask_file:
        print(f"Using hole mask: {os.path.basename(hole_mask_file)}")
    else:
        print("No hole mask found for this sequence")

    try:
        # Load the images
        cadherin_img = tifffile.imread(cadherin_file)
        fused_cell_masks = tifffile.imread(fused_mask_file)

        # Load hole mask if available
        hole_mask = None
        if hole_mask_file:
            hole_mask = tifffile.imread(hole_mask_file)
            print(f"  Hole mask shape: {hole_mask.shape}")

            # Ensure hole mask is binary (holes = 1, background = 0)
            if hole_mask.max() > 1:
                # Convert labeled hole mask to binary
                hole_mask = (hole_mask > 0).astype(np.uint8)

        print(f"  Cadherin image shape: {cadherin_img.shape}")
        print(f"  Fused cell mask shape: {fused_cell_masks.shape}")

        # Extract Cadherin channel if needed
        if len(cadherin_img.shape) == 2:
            # Single channel image (already Cadherin)
            print("  Detected single-channel Cadherin image")
            membrane_channel = cadherin_img
        elif len(cadherin_img.shape) == 3 and cadherin_img.shape[0] == 3:
            # Format is (C, H, W)
            print("  Detected format: (C, H, W)")
            membrane_channel = cadherin_img[0]  # First channel
        elif len(cadherin_img.shape) == 3 and cadherin_img.shape[2] == 3:
            # Format is (H, W, C)
            print("  Detected format: (H, W, C)")
            membrane_channel = cadherin_img[:, :, 0]  # First channel
        else:
            print(f"  Unexpected image shape: {cadherin_img.shape}. Using first channel/plane.")
            if len(cadherin_img.shape) == 3:
                membrane_channel = cadherin_img[0] if cadherin_img.shape[0] < cadherin_img.shape[1] else cadherin_img[:, :, 0]
            else:
                membrane_channel = cadherin_img

        # Apply Gaussian blur to reduce noise
        print("  Applying Gaussian blur...")
        membrane_smoothed = gaussian(membrane_channel, sigma=1)

        # Calculate morphological gradient
        print("  Calculating morphological gradient...")
        selem = disk(1)  # Adjust radius if needed
        dilated = dilation(membrane_smoothed, selem)
        eroded = erosion(membrane_smoothed, selem)
        membrane_gradient = dilated - eroded

        # Normalize gradient to 0-1 range
        membrane_gradient_norm = (membrane_gradient - membrane_gradient.min()) / (
                    membrane_gradient.max() - membrane_gradient.min())

        # Create a valid area mask that excludes holes
        if hole_mask is not None:
            # Resize hole mask if dimensions don't match
            if hole_mask.shape != membrane_gradient_norm.shape:
                hole_mask = resize(hole_mask, membrane_gradient_norm.shape, order=0, preserve_range=True).astype(np.uint8)
                print(f"  Resized hole mask to {hole_mask.shape}")

            # Create valid area mask (areas that are not holes)
            valid_area = (membrane_gradient_norm > 0) & (~hole_mask.astype(bool))
            print("  Created valid area mask excluding holes")

            # Enhance gradient at hole boundaries to create stronger edges
            # This will make watershed avoid crossing hole boundaries
            hole_boundary = dilation(hole_mask, selem) & (~hole_mask.astype(bool))
            membrane_gradient_norm[hole_boundary] = 1.0
            print("  Enhanced gradient at hole boundaries")
        else:
            # If no hole mask, use entire gradient area
            valid_area = membrane_gradient_norm > 0

        # Apply watershed segmentation using fused cell masks as seeds and respecting holes
        print("  Performing watershed segmentation with hole constraints...")
        watershed_output = watershed(membrane_gradient_norm, fused_cell_masks, mask=valid_area)

        # Ensure holes are preserved in the final output
        if hole_mask is not None:
            # Set hole regions to zero in the output
            watershed_output[hole_mask.astype(bool)] = 0
            print("  Ensured holes are preserved in final segmentation")

        # Save the cell mask result
        cell_mask_path = os.path.join(output_dir, f"{base_name}_cell_mask.tif")
        tifffile.imwrite(cell_mask_path, watershed_output.astype(np.uint32))
        print(f"  Saved cell mask to {cell_mask_path}")

        # Print statistics
        print(f"  Number of fused cell seeds: {len(np.unique(fused_cell_masks)) - 1}")
        print(f"  Number of segmented cells: {len(np.unique(watershed_output)) - 1}")
        if hole_mask is not None:
            hole_area_percentage = (hole_mask.sum() / hole_mask.size) * 100
            print(f"  Hole area percentage: {hole_area_percentage:.2f}%")
        print(f"  Processing complete for {filename}")

    except Exception as e:
        print(f"ERROR processing {filename}: {str(e)}")
        continue

print("All processing complete!")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Found 23 Cadherin files to process
Found 23 fused cell mask files
Found 23 hole mask files
Mapped prefix 'denoised_1.4Pa_A1_19dec21_20xA_L2RA_FlatA_seq001' to fused file: denoised_1.4Pa_A1_19dec21_20xA_L2RA_FlatA_seq001_segmented_cells.tif
Mapped prefix 'denoised_1.4Pa_A1_19dec21_20xA_L2RA_FlatA_seq003' to fused file: denoised_1.4Pa_A1_19dec21_20xA_L2RA_FlatA_seq003_segmented_cells.tif
Mapped prefix 'denoised_1.4Pa_A1_19dec21_20xA_L2RA_FlatA_seq002' to fused file: denoised_1.4Pa_A1_19dec21_20xA_L2RA_FlatA_seq002_segmented_cells.tif
Mapped prefix 'denoised_1.4Pa_A1_19dec21_20xA_L2RA_FlatA_seq006' to fused file: denoised_1.4Pa_A1_19dec21_20xA_L2RA_FlatA_seq006_segmented_cells.tif
Mapped prefix 'denoised_1.4Pa_A1_19dec21_20xA_L2RA_FlatA_seq005' to fused file: denoised_1.4Pa_A1_19dec21_20xA_L2RA_FlatA_seq005_segmented_cells.tif
Mapped prefix 'denoised_1.4Pa_A1_19

  0%|          | 0/23 [00:00<?, ?it/s]


Processing: denoised_1.4Pa_A1_19dec21_20xA_L2RA_FlatA_seq001_Cadherins_regional_tophat.tif
Using fused cell mask: denoised_1.4Pa_A1_19dec21_20xA_L2RA_FlatA_seq001_segmented_cells.tif
Using hole mask: denoised_1.4Pa_A1_19dec21_20xA_L2RA_FlatA_seq001_Cadherins_regional_segmented_dilated.tif
  Hole mask shape: (1024, 1024)
  Cadherin image shape: (1024, 1024)
  Fused cell mask shape: (1024, 1024)
  Detected single-channel Cadherin image
  Applying Gaussian blur...
  Calculating morphological gradient...
  Created valid area mask excluding holes
  Enhanced gradient at hole boundaries
  Performing watershed segmentation with hole constraints...
  Ensured holes are preserved in final segmentation
  Saved cell mask to /content/drive/MyDrive/knowledge/University/Master/Thesis/Segmented/1.4Pa-x20/Cell/denoised_1.4Pa_A1_19dec21_20xA_L2RA_FlatA_seq001_cell_mask.tif
  Number of fused cell seeds: 326
  Number of segmented cells: 326
  Hole area percentage: 0.00%
  Processing complete for denoised_

In [ ]:
import numpy as np
import tifffile
from skimage.measure import regionprops, label
from scipy import ndimage
import os
import glob
from tqdm.notebook import tqdm

# Define input and output paths
input_dir = '/content/drive/MyDrive/knowledge/University/Master/Thesis/Segmented/1.4Pa-x20/Cell'
output_dir = '/content/drive/MyDrive/knowledge/University/Master/Thesis/Segmented/1.4Pa-x20/Cell_merged_conservative'

# Create output directory if it doesn't exist
os.makedirs(output_dir, exist_ok=True)

# Find all cell mask files with the correct pattern
cell_mask_files = glob.glob(os.path.join(input_dir, 'denoised_*_cell_mask.tif'))
print(f"Found {len(cell_mask_files)} cell mask files to process")

def check_if_enclave(mask, inner_label, outer_label, min_enclosed_ratio=0.80):
    """
    Check if regions with inner_label are completely enclosed within regions with outer_label.
    Now considers image extremities as part of the boundary.
    """
    # Create binary masks for the specific labels
    inner_mask = mask == inner_label
    outer_mask = mask == outer_label

    # Get the shape of the mask
    height, width = mask.shape

    # Check if the inner region touches any image border
    touches_border = False
    if np.any(inner_mask[0, :]) or np.any(inner_mask[-1, :]) or np.any(inner_mask[:, 0]) or np.any(inner_mask[:, -1]):
        touches_border = True

    # If it touches the border, it's not an enclave
    if touches_border:
        return False

    # Find the boundary of the inner mask using dilation
    dilated_inner = ndimage.binary_dilation(inner_mask)
    inner_boundary = dilated_inner & (~inner_mask)

    # Count how much of the boundary touches the outer mask vs background
    boundary_pixels = np.sum(inner_boundary)
    touching_outer = np.sum(inner_boundary & outer_mask)

    # Calculate the ratio of boundary touching the outer mask
    if boundary_pixels == 0:
        return False

    enclosed_ratio = touching_outer / boundary_pixels

    # Only consider it an enclave if almost all the boundary touches the outer mask
    return enclosed_ratio >= min_enclosed_ratio

def merge_enclaves(mask, max_area_ratio=0.3, min_enclosed_ratio=0.80):
    """
    Find and merge masks that are fully enclosed within other masks.

    Parameters:
    - mask: Labeled mask array
    - max_area_ratio: Maximum area ratio (inner/outer) to consider for merging
    - min_enclosed_ratio: Minimum ratio of boundary touching outer mask

    Returns:
    - Merged mask array
    """
    # Create a copy to avoid modifying the original
    merged_mask = mask.copy()

    # Get region properties
    props = regionprops(merged_mask)

    # Create a dictionary to track label changes
    label_map = {}

    # For each region, check if it's enclosed by another region
    for i, region_i in enumerate(props):
        for j, region_j in enumerate(props):
            if i == j:
                continue

            # Get the bounding boxes
            minr_i, minc_i, maxr_i, maxc_i = region_i.bbox
            minr_j, minc_j, maxr_j, maxc_j = region_j.bbox

            # Quick check: if bounding box of i is not inside bounding box of j, skip
            if not (minr_i >= minr_j and minc_i >= minc_j and
                    maxr_i <= maxr_j and maxc_i <= maxc_j):
                continue

            # Check if inner region is fully enclosed in outer region
            if check_if_enclave(merged_mask, region_i.label, region_j.label, min_enclosed_ratio):
                # Check area ratio if specified
                area_ratio = region_i.area / region_j.area

                # Only merge if the inner region is smaller than the threshold
                if area_ratio <= max_area_ratio:
                    print(f"  Found enclave: Label {region_i.label} inside {region_j.label} (area ratio: {area_ratio:.3f})")
                    label_map[region_i.label] = region_j.label
                else:
                    print(f"  Skipped large region: Label {region_i.label} inside {region_j.label} (area ratio: {area_ratio:.3f} > {max_area_ratio})")

    # Apply the label changes
    for old_label, new_label in label_map.items():
        merged_mask[merged_mask == old_label] = new_label

    # Relabel consecutively without merging all regions
    unique_labels = np.unique(merged_mask[merged_mask > 0])  # Exclude background
    label_mapping = {old: new for new, old in enumerate(unique_labels, start=1)}

    # Create new mask with consecutive labels
    result_mask = np.zeros_like(merged_mask)
    for old_label, new_label in label_mapping.items():
        result_mask[merged_mask == old_label] = new_label

    return result_mask


# Process each cell mask file
for mask_file in tqdm(cell_mask_files):
    filename = os.path.basename(mask_file)
    print(f"\nProcessing: {filename}")

    try:
        # Load the cell mask
        cell_mask = tifffile.imread(mask_file)

        # Get initial statistics
        initial_labels = np.unique(cell_mask)
        initial_count = len(initial_labels) - 1  # Subtract 1 for background
        print(f"  Initial number of cells: {initial_count}")

        # Merge enclaves with conservative parameters - only merge small enclaves
        merged_mask = merge_enclaves(cell_mask, max_area_ratio=0.3, min_enclosed_ratio=0.80)

        # Get final statistics
        final_labels = np.unique(merged_mask)
        final_count = len(final_labels) - 1  # Subtract 1 for background
        print(f"  Final number of cells: {final_count}")
        print(f"  Merged {initial_count - final_count} enclaves")

        # Save the merged mask
        # Remove 'denoised_' prefix and modify the suffix
        output_filename = filename.replace('denoised_', '').replace('_cell_mask.tif', '_cell_mask_merged_conservative.tif')
        output_path = os.path.join(output_dir, output_filename)
        tifffile.imwrite(output_path, merged_mask.astype(np.uint32))
        print(f"  Saved merged mask to {output_path}")

    except Exception as e:
        print(f"ERROR processing {filename}: {str(e)}")
        continue

print("\nAll processing complete!")

Found 23 cell mask files to process


  0%|          | 0/23 [00:00<?, ?it/s]


Processing: denoised_1.4Pa_A1_19dec21_20xA_L2RA_FlatA_seq001_cell_mask.tif
  Initial number of cells: 326
  Final number of cells: 326
  Merged 0 enclaves
  Saved merged mask to /content/drive/MyDrive/knowledge/University/Master/Thesis/Segmented/1.4Pa-x20/Cell_merged_conservative/1.4Pa_A1_19dec21_20xA_L2RA_FlatA_seq001_cell_mask_merged_conservative.tif

Processing: denoised_1.4Pa_A1_19dec21_20xA_L2RA_FlatA_seq002_cell_mask.tif
  Initial number of cells: 276
  Found enclave: Label 19 inside 37 (area ratio: 0.055)
  Found enclave: Label 105 inside 111 (area ratio: 0.184)
  Found enclave: Label 266 inside 259 (area ratio: 0.186)
  Final number of cells: 273
  Merged 3 enclaves
  Saved merged mask to /content/drive/MyDrive/knowledge/University/Master/Thesis/Segmented/1.4Pa-x20/Cell_merged_conservative/1.4Pa_A1_19dec21_20xA_L2RA_FlatA_seq002_cell_mask_merged_conservative.tif

Processing: denoised_1.4Pa_A1_19dec21_20xA_L2RA_FlatA_seq004_cell_mask.tif
  Initial number of cells: 276
  Final n

In [ ]:
import numpy as np
import tifffile
from skimage.measure import regionprops, label
from scipy import ndimage
import matplotlib.pyplot as plt
import os
import glob
from google.colab import drive
from tqdm.notebook import tqdm

# Mount Google Drive if not already mounted
drive.mount('/content/drive', force_remount=False)

# Define paths
input_dir = '/content/drive/MyDrive/knowledge/University/Master/Thesis/Segmented/1.4Pa-x20/Cell'
merged_dir = '/content/drive/MyDrive/knowledge/University/Master/Thesis/Segmented/1.4Pa-x20/Cell_merged_conservative'
visualization_dir = '/content/drive/MyDrive/knowledge/University/Master/Thesis/Segmented/1.4Pa-x20/Conservative_merge_visualization'

# Create visualization directory if it doesn't exist
os.makedirs(visualization_dir, exist_ok=True)

# Find all cell mask files - original ones (with denoised_ prefix)
# First try with .tif extension
cell_mask_files = glob.glob(os.path.join(input_dir, 'denoised_*_cell_mask.tif'))
if len(cell_mask_files) == 0:
    # If none found, try without extension
    cell_mask_files = glob.glob(os.path.join(input_dir, 'denoised_*_cell_mask'))
print(f"Found {len(cell_mask_files)} cell mask files to process")

def check_if_enclave(mask, inner_label, outer_label, min_enclosed_ratio=0.80):
    """
    Check if regions with inner_label are completely enclosed within regions with outer_label.
    Now considers image extremities as part of the boundary.
    """
    # Create binary masks for the specific labels
    inner_mask = mask == inner_label
    outer_mask = mask == outer_label

    # Get the shape of the mask
    height, width = mask.shape

    # Check if the inner region touches any image border
    touches_border = False
    if np.any(inner_mask[0, :]) or np.any(inner_mask[-1, :]) or np.any(inner_mask[:, 0]) or np.any(inner_mask[:, -1]):
        touches_border = True

    # If it touches the border, it's not an enclave
    if touches_border:
        return False

    # Find the boundary of the inner mask using dilation
    dilated_inner = ndimage.binary_dilation(inner_mask)
    inner_boundary = dilated_inner & (~inner_mask)

    # Count how much of the boundary touches the outer mask vs background
    boundary_pixels = np.sum(inner_boundary)
    touching_outer = np.sum(inner_boundary & outer_mask)

    # Calculate the ratio of boundary touching the outer mask
    if boundary_pixels == 0:
        return False

    enclosed_ratio = touching_outer / boundary_pixels
    return enclosed_ratio >= min_enclosed_ratio

def visualize_merges(original_mask, merged_mask, filename, max_area_ratio=0.3):
    """
    Visualize only the regions that were merged
    """
    # Find which regions were merged
    original_props = regionprops(original_mask)
    merged_props = regionprops(merged_mask)

    # Create label mapping from original to merged
    label_map = {}
    enclave_pairs = []

    # Check each pair of regions to find enclaves
    for i, region_i in enumerate(original_props):
        for j, region_j in enumerate(original_props):
            if i == j:
                continue

            minr_i, minc_i, maxr_i, maxc_i = region_i.bbox
            minr_j, minc_j, maxr_j, maxc_j = region_j.bbox

            if not (minr_i >= minr_j and minc_i >= minc_j and
                    maxr_i <= maxr_j and maxc_i <= maxc_j):
                continue

            if check_if_enclave(original_mask, region_i.label, region_j.label):
                area_ratio = region_i.area / region_j.area
                if area_ratio <= max_area_ratio:
                    enclave_pairs.append((region_i, region_j))
                    label_map[region_i.label] = region_j.label

    # For each enclave pair, create a before/after visualization
    for idx, (inner_region, outer_region) in enumerate(enclave_pairs):
        fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 6))

        # Calculate bounding box that includes both regions with some padding
        padding = 20
        minr = max(0, outer_region.bbox[0] - padding)
        minc = max(0, outer_region.bbox[1] - padding)
        maxr = min(original_mask.shape[0], outer_region.bbox[2] + padding)
        maxc = min(original_mask.shape[1], outer_region.bbox[3] + padding)

        # Extract the regions of interest
        original_roi = original_mask[minr:maxr, minc:maxc]
        merged_roi = merged_mask[minr:maxr, minc:maxc]

        # Create visualization masks
        before_vis = np.zeros((*original_roi.shape, 3))
        after_vis = np.zeros((*merged_roi.shape, 3))

        # Highlight the outer region in blue, inner region in red (before)
        before_vis[original_roi == outer_region.label] = [0, 0, 1]  # Blue
        before_vis[original_roi == inner_region.label] = [1, 0, 0]  # Red

        # Find the new label in the merged mask
        merged_label_at_position = merged_mask[inner_region.coords[0][0], inner_region.coords[0][1]]

        # Highlight the merged region in purple (after)
        after_vis[merged_roi == merged_label_at_position] = [0.5, 0, 0.5]  # Purple

        # Calculate the area ratio for display
        area_ratio = inner_region.area / outer_region.area

        # Plot before
        ax1.imshow(before_vis)
        ax1.set_title(f'Before: Outer (blue) #{outer_region.label}, Inner (red) #{inner_region.label}\nArea ratio: {area_ratio:.3f}')
        ax1.axis('off')

        # Plot after
        ax2.imshow(after_vis)
        ax2.set_title(f'After: Merged into #{merged_label_at_position}')
        ax2.axis('off')

        plt.suptitle(f'{filename} - Merge {idx+1}', fontsize=14)
        plt.tight_layout()

        # Save the figure
        save_path = os.path.join(visualization_dir, f"{filename}_merge_{idx+1}.png")
        plt.savefig(save_path, dpi=150, bbox_inches='tight')
        plt.close()

    return len(enclave_pairs)

# Also create a summary visualization showing all masks side by side
def create_summary_visualization(original_mask, merged_mask, filename):
    """
    Create a side-by-side comparison of the full original and merged masks
    """
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 8))

    # Create color maps for visualization
    from matplotlib.colors import ListedColormap
    import matplotlib.cm as cm

    # Generate a colormap with enough colors for all labels
    n_labels = max(np.max(original_mask), np.max(merged_mask)) + 1
    cmap = cm.get_cmap('tab20', n_labels)

    # Plot original mask
    ax1.imshow(original_mask, cmap=cmap)
    ax1.set_title(f'Original Segmentation\n{len(np.unique(original_mask)) - 1} cells')
    ax1.axis('off')

    # Plot merged mask
    ax2.imshow(merged_mask, cmap=cmap)
    ax2.set_title(f'After Conservative Merging\n{len(np.unique(merged_mask)) - 1} cells')
    ax2.axis('off')

    plt.suptitle(f'{filename}', fontsize=14)
    plt.tight_layout()

    # Save the figure
    save_path = os.path.join(visualization_dir, f"{filename}_summary.png")
    plt.savefig(save_path, dpi=150, bbox_inches='tight')
    plt.close()

# Process each pair of original and merged masks
for mask_file in tqdm(cell_mask_files):
    filename = os.path.basename(mask_file)
    base_name = filename.replace('_cell_mask.tif', '')

    # Handle the denoised_ prefix issue
    if filename.startswith('denoised_'):
        # Remove denoised_ prefix and handle both .tif and no extension
        if filename.endswith('.tif'):
            common_part = filename.replace('denoised_', '').replace('_cell_mask.tif', '')
            merged_filename = f"{common_part}_cell_mask_merged_conservative.tif"
        else:
            common_part = filename.replace('denoised_', '').replace('_cell_mask', '')
            merged_filename = f"{common_part}_cell_mask_merged_conservative"
    else:
        if filename.endswith('.tif'):
            merged_filename = filename.replace('_cell_mask.tif', '_cell_mask_merged_conservative.tif')
        else:
            merged_filename = filename.replace('_cell_mask', '_cell_mask_merged_conservative')

    merged_file = os.path.join(merged_dir, merged_filename)

    if not os.path.exists(merged_file):
        print(f"WARNING: No merged mask found for {filename}")
        print(f"  Expected merged file: {merged_filename}")
        continue

    print(f"\nProcessing: {filename}")

    try:
        # Load both masks
        original_mask = tifffile.imread(mask_file)
        merged_mask = tifffile.imread(merged_file)

        # Create summary visualization
        create_summary_visualization(original_mask, merged_mask, base_name)

        # Visualize the individual merges
        num_merges = visualize_merges(original_mask, merged_mask, base_name)
        print(f"  Created {num_merges} merge visualizations and 1 summary visualization")

    except Exception as e:
        print(f"ERROR processing {filename}: {str(e)}")
        continue

print(f"\nAll visualizations saved to: {visualization_dir}")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Found 23 cell mask files to process


  0%|          | 0/23 [00:00<?, ?it/s]


Processing: denoised_1.4Pa_A1_19dec21_20xA_L2RA_FlatA_seq001_cell_mask.tif


<ipython-input-3-a770bc0c657c>:162: MatplotlibDeprecationWarning: The get_cmap function was deprecated in Matplotlib 3.7 and will be removed in 3.11. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap()`` or ``pyplot.get_cmap()`` instead.
  cmap = cm.get_cmap('tab20', n_labels)


  Created 0 merge visualizations and 1 summary visualization

Processing: denoised_1.4Pa_A1_19dec21_20xA_L2RA_FlatA_seq002_cell_mask.tif
  Created 3 merge visualizations and 1 summary visualization

Processing: denoised_1.4Pa_A1_19dec21_20xA_L2RA_FlatA_seq004_cell_mask.tif
  Created 0 merge visualizations and 1 summary visualization

Processing: denoised_1.4Pa_A1_19dec21_20xA_L2RA_FlatA_seq003_cell_mask.tif
  Created 0 merge visualizations and 1 summary visualization

Processing: denoised_1.4Pa_A1_19dec21_20xA_L2RA_FlatA_seq007_cell_mask.tif
  Created 0 merge visualizations and 1 summary visualization

Processing: denoised_1.4Pa_A1_19dec21_20xA_L2RA_FlatA_seq006_cell_mask.tif
  Created 1 merge visualizations and 1 summary visualization

Processing: denoised_1.4Pa_A1_19dec21_20xA_L2RA_FlatA_seq005_cell_mask.tif
  Created 0 merge visualizations and 1 summary visualization

Processing: denoised_1.4Pa_A1_19dec21_20xA_L2RA_FlatA_seq008_cell_mask.tif
  Created 0 merge visualizations and 1 su

In [ ]:
#before
import numpy as np
import tifffile
from skimage.measure import regionprops, label
from scipy import ndimage
import matplotlib.pyplot as plt
import os
import glob
from google.colab import drive
from tqdm.notebook import tqdm

# Mount Google Drive if not already mounted
drive.mount('/content/drive', force_remount=False)

# Define paths
input_dir = '/content/drive/MyDrive/knowledge/University/Master/Thesis/Segmented/1.4Pa-x20/Cell'
merged_dir = '/content/drive/MyDrive/knowledge/University/Master/Thesis/Segmented/1.4Pa-x20/Cell_merged_conservative'
visualization_dir = '/content/drive/MyDrive/knowledge/University/Master/Thesis/Segmented/1.4Pa-x20/Conservative_merge_visualization'

# Create visualization directory if it doesn't exist
os.makedirs(visualization_dir, exist_ok=True)

# Find all cell mask files
cell_mask_files = glob.glob(os.path.join(input_dir, '*_cell_mask.tif'))
print(f"Found {len(cell_mask_files)} cell mask files to process")

def check_if_enclave(mask, inner_label, outer_label, min_enclosed_ratio=0.80):
    """
    Check if regions with inner_label are completely enclosed within regions with outer_label
    """
    inner_mask = mask == inner_label
    outer_mask = mask == outer_label

    dilated_inner = ndimage.binary_dilation(inner_mask)
    inner_boundary = dilated_inner & (~inner_mask)

    boundary_pixels = np.sum(inner_boundary)
    touching_outer = np.sum(inner_boundary & outer_mask)

    if boundary_pixels == 0:
        return False

    enclosed_ratio = touching_outer / boundary_pixels
    return enclosed_ratio >= min_enclosed_ratio

def visualize_merges(original_mask, merged_mask, filename, max_area_ratio=0.3):
    """
    Visualize only the regions that were merged
    """
    # Find which regions were merged
    original_props = regionprops(original_mask)
    merged_props = regionprops(merged_mask)

    # Create label mapping from original to merged
    label_map = {}
    enclave_pairs = []

    # Check each pair of regions to find enclaves
    for i, region_i in enumerate(original_props):
        for j, region_j in enumerate(original_props):
            if i == j:
                continue

            minr_i, minc_i, maxr_i, maxc_i = region_i.bbox
            minr_j, minc_j, maxr_j, maxc_j = region_j.bbox

            if not (minr_i >= minr_j and minc_i >= minc_j and
                    maxr_i <= maxr_j and maxc_i <= maxc_j):
                continue

            if check_if_enclave(original_mask, region_i.label, region_j.label):
                area_ratio = region_i.area / region_j.area
                if area_ratio <= max_area_ratio:
                    enclave_pairs.append((region_i, region_j))
                    label_map[region_i.label] = region_j.label

    # For each enclave pair, create a before/after visualization
    for idx, (inner_region, outer_region) in enumerate(enclave_pairs):
        fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 6))

        # Calculate bounding box that includes both regions with some padding
        padding = 20
        minr = max(0, outer_region.bbox[0] - padding)
        minc = max(0, outer_region.bbox[1] - padding)
        maxr = min(original_mask.shape[0], outer_region.bbox[2] + padding)
        maxc = min(original_mask.shape[1], outer_region.bbox[3] + padding)

        # Extract the regions of interest
        original_roi = original_mask[minr:maxr, minc:maxc]
        merged_roi = merged_mask[minr:maxr, minc:maxc]

        # Create visualization masks
        before_vis = np.zeros((*original_roi.shape, 3))
        after_vis = np.zeros((*merged_roi.shape, 3))

        # Highlight the outer region in blue, inner region in red (before)
        before_vis[original_roi == outer_region.label] = [0, 0, 1]  # Blue
        before_vis[original_roi == inner_region.label] = [1, 0, 0]  # Red

        # Find the new label in the merged mask
        merged_label_at_position = merged_mask[inner_region.coords[0][0], inner_region.coords[0][1]]

        # Highlight the merged region in purple (after)
        after_vis[merged_roi == merged_label_at_position] = [0.5, 0, 0.5]  # Purple

        # Calculate the area ratio for display
        area_ratio = inner_region.area / outer_region.area

        # Plot before
        ax1.imshow(before_vis)
        ax1.set_title(f'Before: Outer (blue) #{outer_region.label}, Inner (red) #{inner_region.label}\nArea ratio: {area_ratio:.3f}')
        ax1.axis('off')

        # Plot after
        ax2.imshow(after_vis)
        ax2.set_title(f'After: Merged into #{merged_label_at_position}')
        ax2.axis('off')

        plt.suptitle(f'{filename} - Merge {idx+1}', fontsize=14)
        plt.tight_layout()

        # Save the figure
        save_path = os.path.join(visualization_dir, f"{filename}_merge_{idx+1}.png")
        plt.savefig(save_path, dpi=150, bbox_inches='tight')
        plt.close()

    return len(enclave_pairs)

# Also create a summary visualization showing all masks side by side
def create_summary_visualization(original_mask, merged_mask, filename):
    """
    Create a side-by-side comparison of the full original and merged masks
    """
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 8))

    # Create color maps for visualization
    from matplotlib.colors import ListedColormap
    import matplotlib.cm as cm

    # Generate a colormap with enough colors for all labels
    n_labels = max(np.max(original_mask), np.max(merged_mask)) + 1
    cmap = cm.get_cmap('tab20', n_labels)

    # Plot original mask
    ax1.imshow(original_mask, cmap=cmap)
    ax1.set_title(f'Original Segmentation\n{len(np.unique(original_mask)) - 1} cells')
    ax1.axis('off')

    # Plot merged mask
    ax2.imshow(merged_mask, cmap=cmap)
    ax2.set_title(f'After Conservative Merging\n{len(np.unique(merged_mask)) - 1} cells')
    ax2.axis('off')

    plt.suptitle(f'{filename}', fontsize=14)
    plt.tight_layout()

    # Save the figure
    save_path = os.path.join(visualization_dir, f"{filename}_summary.png")
    plt.savefig(save_path, dpi=150, bbox_inches='tight')
    plt.close()

# Process each pair of original and merged masks
for mask_file in tqdm(cell_mask_files):
    filename = os.path.basename(mask_file)
    base_name = filename.replace('_cell_mask.tif', '')

    # Find corresponding merged mask
    merged_filename = filename.replace('_cell_mask.tif', '_cell_mask_merged_conservative.tif')
    merged_file = os.path.join(merged_dir, merged_filename)

    if not os.path.exists(merged_file):
        print(f"WARNING: No merged mask found for {filename}")
        continue

    print(f"\nProcessing: {filename}")

    try:
        # Load both masks
        original_mask = tifffile.imread(mask_file)
        merged_mask = tifffile.imread(merged_file)

        # Create summary visualization
        create_summary_visualization(original_mask, merged_mask, base_name)

        # Visualize the individual merges
        num_merges = visualize_merges(original_mask, merged_mask, base_name)
        print(f"  Created {num_merges} merge visualizations and 1 summary visualization")

    except Exception as e:
        print(f"ERROR processing {filename}: {str(e)}")
        continue

print(f"\nAll visualizations saved to: {visualization_dir}")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Found 23 cell mask files to process


  0%|          | 0/23 [00:00<?, ?it/s]


Processing: denoised_1.4Pa_A1_19dec21_20xA_L2RA_FlatA_seq001_cell_mask.tif


<ipython-input-12-01bbd41099d8>:141: MatplotlibDeprecationWarning: The get_cmap function was deprecated in Matplotlib 3.7 and will be removed in 3.11. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap()`` or ``pyplot.get_cmap()`` instead.
  cmap = cm.get_cmap('tab20', n_labels)


  Created 0 merge visualizations and 1 summary visualization

Processing: denoised_1.4Pa_A1_19dec21_20xA_L2RA_FlatA_seq002_cell_mask.tif
  Created 3 merge visualizations and 1 summary visualization

Processing: denoised_1.4Pa_A1_19dec21_20xA_L2RA_FlatA_seq003_cell_mask.tif
  Created 1 merge visualizations and 1 summary visualization

Processing: denoised_1.4Pa_A1_19dec21_20xA_L2RA_FlatA_seq004_cell_mask.tif
  Created 1 merge visualizations and 1 summary visualization

Processing: denoised_1.4Pa_A1_19dec21_20xA_L2RA_FlatA_seq005_cell_mask.tif
  Created 0 merge visualizations and 1 summary visualization

Processing: denoised_1.4Pa_A1_19dec21_20xA_L2RA_FlatA_seq006_cell_mask.tif
  Created 1 merge visualizations and 1 summary visualization

Processing: denoised_1.4Pa_A1_19dec21_20xA_L2RA_FlatA_seq007_cell_mask.tif
  Created 0 merge visualizations and 1 summary visualization

Processing: denoised_1.4Pa_A1_19dec21_20xA_L2RA_FlatA_seq008_cell_mask.tif
  Created 0 merge visualizations and 1 su